In [1]:
import pandas as pd

In [2]:
APP_NAME = "nishanth-test"

try:
    sc.stop()
    spark.stop()
except:
    pass

from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

try:
    sc and spark
except (NameError, UnboundLocalError) as e:
    import findspark
    #findspark.init()
    import pyspark
    import pyspark.sql
    
    #sc = pyspark.SparkContext()
    #spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()
    
sc = SparkContext(conf = SparkConf() .set("spark.driver.maxResultSize", "20g") 
                  .set("spark.sql.execution.arrow.enabled", "true") 
                  .set('spark.sql.broadcastTimeout', 1000) 
                  .set('spark.local.dir', '/data_data/session_length/nishanth01/tmp/') 
                  .set('spark.driver.memory', '20G') 
                  .set("spark.executor.instances", "20") 
                  .set("spark.executor.cores", 20) 
                  .set("spark.executor.memory", "8G")).getOrCreate()
spark = SparkSession(sc)
spark.sparkContext.setLogLevel("ERROR")

In [3]:
sc._conf.getAll()

[(u'spark.app.id', u'local-1538717170730'),
 (u'spark.driver.memory', u'20G'),
 (u'spark.sql.execution.arrow.enabled', u'true'),
 (u'spark.executor.cores', u'20'),
 (u'spark.executor.memory', u'8G'),
 (u'spark.executor.id', u'driver'),
 (u'spark.app.name', u'pyspark-shell'),
 (u'spark.executor.instances', u'20'),
 (u'spark.driver.maxResultSize', u'20g'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.host', u'ip-172-31-41-62.ec2.internal'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.sql.broadcastTimeout', u'1000'),
 (u'spark.driver.port', u'44295'),
 (u'spark.ui.showConsoleProgress', u'true'),
 (u'spark.local.dir', u'/data_data/session_length/nishanth01/tmp/')]

In [4]:
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.functions import when
from pyspark.sql.types import StructField, StructType, StringType, LongType, DateType, DoubleType, IntegerType
from pyspark.sql.functions import count, mean, stddev_pop, min, max, lit, round, bround, pow, col, corr, lower, upper, avg, stddev, abs, log
from pyspark.sql.functions import lit, trim, rtrim, rpad, trim, coalesce
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, months_between, to_date
from pyspark.sql.functions import udf, col, countDistinct, lag
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, rank, expr

from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula, Bucketizer, QuantileDiscretizer
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier, RandomForestClassifier, MultilayerPerceptronClassifier,NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.linalg import Vectors

from datetime import datetime as dt
from datetime import timedelta
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os

In [5]:
!pwd

/home/nishanth01


In [6]:
os.chdir("/data_data/session_length/nishanth01")

In [7]:
!pwd

/data_data/session_length/nishanth01


In [8]:
!head data/user_session_original.tsv

user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)
user_000001	2009-05-04T13:42:11Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc1 (Live_2009_4_15)
user_000001	2009-05-04T13:38:31Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		To Stanford (Live_2009_4_15)
user_000001	2009-05-04T13:33:28Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Improvisation (Live_2009_4_15)
user_000001	2009-05-04T13:23:45Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Glacier (Live_2009_4_15)
user_000001	2009-05-04T13:19:22Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Parolibre (Live_2009_4_15)
user_000001	2009-05-04T13:13:38Z	a7f7df4a-77d8-4f12-8acd

In [9]:
session_file = '/data_data/session_length/nishanth01/data/user_session_original.tsv'
profile_file = '/data_data/session_length/nishanth01/data/userid-profile.csv'   
tracks_file = '/data_data/session_length/nishanth01/data/tracks/tracks_complete_*.csv'
genres_file = '/data_data/session_length/nishanth01/data/genres.csv'
users_file = '/data_data/session_length/nishanth01/data/unique_users.txt'

In [10]:
# with open(session_file) as tsvin
#     tsvin = csv.reader(tsvin, delimiter='\t')
#     for row in tqdm(tsvin,total=19150868):
#         try:
#             row = [x.strip().lower() for x in row]
#         except Exception:
#             print('EXCEPTION: ',row)
#             passs

In [11]:
def transform(self, f):
    return f(self)

DataFrame.transform = transform

def get_user_session_schema():
        sessionSchema = StructType([
        StructField('user_id', StringType(), True),\
        StructField('timestamp', StringType(), True),\
        StructField('artist_id', StringType(), True),\
        StructField('artist_name', StringType(), True),\
        StructField('track_id', StringType(), True),\
        StructField('track_name', StringType(), True),\
        ])
        return sessionSchema
    

def get_profile_schema():
        sessionSchema = StructType([
        StructField('user_id', StringType(), True),\
        StructField('gender', StringType(), True),\
        StructField('age', IntegerType(), True),\
        StructField('country', StringType(), True),\
        StructField('registered', StringType(), True),\
        ])
        return sessionSchema

    
def get_tracks_schema():
    profileSchema = StructType([
        StructField('artist_id', StringType(), True),\
        StructField('artist_name', StringType(), True),\
        StructField('track_name', StringType(), True),\
        StructField('count', IntegerType(), True),\
        StructField('duration', IntegerType(), True),\
        StructField('genre', StringType(), True),\
        ])
    return profileSchema


def get_current_age(timestamp,registered,age):
    current_age = 0
    try:
        time = dt.strptime(timestamp,'%Y-%m-%dt%H:%M:%Sz')
        current_year = time.strftime('%Y')
        registered_date = dt.strptime(registered,'%b %d, %Y')
        registered_year = registered_date.strftime('%Y')
        if(age is not None and age is not ''):
            current_age = int(age) + (int(current_year) - int(registered_year))
    except Exception as e:
        current_age = 0
        pass
    return current_age     


compute_age = udf(lambda timestamp,registered,age : get_current_age(str(timestamp),str(registered),age))

def fix_age():
    def inner(df):
        return df.withColumn('age',compute_age(df.timestamp,df.registered,df.age))
    return inner


def get_user_session():
    user_sessions_df = (spark.read.format('com.databricks.spark.csv')
                        .schema(get_user_session_schema())
                        .option('header',False)
                        .option('sep','\t')
                        .load(session_file))
    
    user_profile_df = (spark.read.format('com.databricks.spark.csv')
                       .schema(get_profile_schema())
                       .option('header',False)
                       .load(profile_file))
    
    tracks_df = (spark.read.format('com.databricks.spark.csv')
                 .schema(get_tracks_schema())
                 .option('header',False)
                 .load(tracks_file))
   
    df1 = (user_sessions_df.alias('a')
            .join(user_profile_df.alias('b'),
                 col('a.user_id') == col('b.user_id'),
                  how='left')
           .select(['a.user_id','a.timestamp',
                    'a.artist_name','a.track_name',
                    'b.gender','b.age','b.country',
                    'b.registered']))
    
    df2 = (df1.alias('a')
            .join(tracks_df.alias('b'),
                 (lower(col('a.track_name')) == lower(col('b.track_name'))) & 
                   (lower(col('a.artist_name')) == lower(col('b.artist_name'))),
                     how='left')
           .select(['a.user_id','a.timestamp','a.artist_name',
                    'a.track_name','a.gender','a.age',
                    'a.country','a.registered',
                    coalesce('b.duration',lit(0)).alias('duration'),
                    coalesce('b.genre',lit('')).alias('genre')]))
    
    df3 = (df2.transform(fix_age()))    
    
    return df3


def get_users():
    columns = ['user_id']
    users = pd.read_csv(users_file,names=columns)
    return users


def create_user(user_id,data):
    try:
        new_dir = '/data_data/session_length/nishanth01/data/users/{0}'.format(user_id)
        final = data.where(col('user_id') == user_id)
        (final.write
             .option("sep","\t")
             .format('com.databricks.spark.csv')
             .save(new_dir))        
        print('COMPLETED: {0}'.format(user_id))   
    except Exception as e:
        print(e)
        pass

def process(df):
    print('Starting..')
    completed = os.listdir('/data_data/session_length/nishanth01/data/users/')
    try:
        i = 0
        users = get_users()
        for row in users.iterrows():
            user_id = row[1]['user_id']
            if(user_id not in completed):
                create_user(user_id,df)
            i += 1
#             if(i == 4):
#                 break
    except Exception as e:
        print('EXCEPTION 0 :::',e)
    print('COMPLETE!')

    

In [12]:
!rm -rf data/user_session_complete
!rm -rf data/user_session_complete.csv
!rm -rf data/users/*

In [13]:
process(get_user_session())

Starting..
COMPLETED: user_000001
COMPLETED: user_000002
COMPLETED: user_000003
COMPLETED: user_000004
COMPLETED: user_000005
COMPLETED: user_000006
COMPLETED: user_000007
COMPLETED: user_000008
COMPLETED: user_000009
COMPLETED: user_000010
COMPLETED: user_000011
COMPLETED: user_000012
COMPLETED: user_000013
COMPLETED: user_000014
COMPLETED: user_000015
COMPLETED: user_000016
COMPLETED: user_000017
COMPLETED: user_000018
COMPLETED: user_000019
COMPLETED: user_000020
COMPLETED: user_000021
COMPLETED: user_000022
COMPLETED: user_000023
COMPLETED: user_000024
COMPLETED: user_000025
COMPLETED: user_000026
COMPLETED: user_000027
COMPLETED: user_000028
COMPLETED: user_000029
COMPLETED: user_000030
COMPLETED: user_000031
COMPLETED: user_000032
COMPLETED: user_000033
COMPLETED: user_000034
COMPLETED: user_000035
COMPLETED: user_000036
COMPLETED: user_000037
COMPLETED: user_000038
COMPLETED: user_000039
COMPLETED: user_000040
COMPLETED: user_000041
COMPLETED: user_000042
COMPLETED: user_000043


COMPLETED: user_000359
COMPLETED: user_000360
COMPLETED: user_000361
COMPLETED: user_000362
COMPLETED: user_000363
COMPLETED: user_000364
COMPLETED: user_000366
COMPLETED: user_000367
COMPLETED: user_000368
COMPLETED: user_000369
COMPLETED: user_000370
COMPLETED: user_000371
COMPLETED: user_000372
COMPLETED: user_000373
COMPLETED: user_000374
COMPLETED: user_000375
COMPLETED: user_000376
COMPLETED: user_000377
COMPLETED: user_000378
COMPLETED: user_000379
COMPLETED: user_000380
COMPLETED: user_000381
COMPLETED: user_000382
COMPLETED: user_000383
COMPLETED: user_000384
COMPLETED: user_000385
COMPLETED: user_000386
COMPLETED: user_000387
COMPLETED: user_000388
COMPLETED: user_000389
COMPLETED: user_000390
COMPLETED: user_000391
COMPLETED: user_000392
COMPLETED: user_000393
COMPLETED: user_000394
COMPLETED: user_000395
COMPLETED: user_000396
COMPLETED: user_000397
COMPLETED: user_000398
COMPLETED: user_000399
COMPLETED: user_000400
COMPLETED: user_000401
COMPLETED: user_000402
COMPLETED: 

COMPLETED: user_000721
COMPLETED: user_000722
COMPLETED: user_000723
COMPLETED: user_000724
COMPLETED: user_000725
COMPLETED: user_000726
COMPLETED: user_000727
COMPLETED: user_000728
COMPLETED: user_000729
COMPLETED: user_000730
COMPLETED: user_000731
COMPLETED: user_000732
COMPLETED: user_000733
COMPLETED: user_000734
COMPLETED: user_000735
COMPLETED: user_000736
COMPLETED: user_000737
COMPLETED: user_000738
COMPLETED: user_000739
COMPLETED: user_000740
COMPLETED: user_000741
COMPLETED: user_000742
COMPLETED: user_000743
COMPLETED: user_000744
COMPLETED: user_000745
COMPLETED: user_000746
COMPLETED: user_000747
COMPLETED: user_000748
COMPLETED: user_000749
COMPLETED: user_000750
COMPLETED: user_000751
COMPLETED: user_000752
COMPLETED: user_000753
COMPLETED: user_000754
COMPLETED: user_000755
COMPLETED: user_000756
COMPLETED: user_000757
COMPLETED: user_000758
COMPLETED: user_000759
COMPLETED: user_000760
COMPLETED: user_000761
COMPLETED: user_000762
COMPLETED: user_000763
COMPLETED: 